In [1]:
import sys
from dotenv import load_dotenv
import os
import requests
import pandas as pd
import json

In [2]:
load_dotenv()
load_dotenv('.env')

True

In [3]:

sports_data_api_key = os.getenv('SPORTS_DATA_API_KEY')
url = "https://api-nba-v1.p.rapidapi.com/players"
api_key_str = '?key='+sports_data_api_key
querystring = {"team":"9","season":"2023"}

headers = {
	"X-RapidAPI-Key": sports_data_api_key,
	"X-RapidAPI-Host": "api-nba-v1.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)
response_nuggets = response.json()
print(json.dumps(response_nuggets, indent=4))

{
    "get": "players/",
    "parameters": {
        "team": "9",
        "season": "2023"
    },
    "errors": [],
    "results": 22,
    "response": [
        {
            "id": 383,
            "firstname": "Jamal",
            "lastname": "Murray",
            "birth": {
                "date": "1997-02-23",
                "country": "Canada"
            },
            "nba": {
                "start": 2016,
                "pro": 5
            },
            "height": {
                "feets": "6",
                "inches": "3",
                "meters": "1.9"
            },
            "weight": {
                "pounds": "215",
                "kilograms": "97.5"
            },
            "college": "Kentucky",
            "affiliation": "Kentucky/Canada",
            "leagues": {
                "standard": {
                    "jersey": 27,
                    "active": true,
                    "pos": "G"
                }
            }
        },
        {
            

In [4]:
nuggets_df = pd.json_normalize(response_nuggets['response']).reset_index(drop=True)
nuggets_df

,id,firstname,lastname,college,affiliation,birth.date,birth.country,nba.start,nba.pro,height.feets,...,leagues.standard.pos,leagues.vegas.jersey,leagues.vegas.active,leagues.vegas.pos,leagues.utah.jersey,leagues.utah.active,leagues.utah.pos,leagues.sacramento.jersey,leagues.sacramento.active,leagues.sacramento.pos
0,383,Jamal,Murray,Kentucky,Kentucky/Canada,1997-02-23,Canada,2016,5,6,...,G,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1055,Amida,Brimah,Connecticut,Connecticut/USA,1994-02-11,USA,2020,1,6,...,C,37.0,True,C,37.0,True,C,NaN,NaN,NaN
2,1014,Michael,Porter Jr.,Missouri,Missouri/USA,1998-06-29,USA,2019,2,6,...,F,1.0,True,F,NaN,NaN,NaN,NaN,NaN,NaN
3,2627,Zeke,Nnaji,Arizona,Arizona/USA,2001-01-09,USA,2020,1,6,...,F-C,22.0,True,F,NaN,NaN,NaN,NaN,NaN,NaN
4,3091,Braxton,Key,Virginia,Virginia/USA,1997-02-14,USA,2021,0,6,...,F,8.0,True,F,NaN,NaN,NaN,NaN,NaN,NaN
5,2814,Jay,Huff,Virginia,Virginia/USA,1997-08-25,USA,2021,0,None,...,F,30.0,True,C,NaN,NaN,NaN,30.0,True,C
6,2840,Jamorko,Pickett,Georgetown,Georgetown/USA,1997-12-24,USA,2021,0,6,...,F,45.0,True,F,NaN,NaN,NaN,NaN,NaN,NaN
7,3420,Christian,Braun,None,None,None,None,0,0,None,...,G,NaN,True,F,NaN,NaN,NaN,NaN,NaN,NaN
8,3498,Peyton,Watson,None,None,None,None,0,0,None,...,G,8.0,True,F,NaN,NaN,NaN,NaN,NaN,NaN
9,3441,Collin,Gillespie,None,None,None,None,0,0,None,...,G,21.0,True,G,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:

url = "https://api-nba-v1.p.rapidapi.com/games"

querystring = {"season":"2023","team":"9"}

headers = {
	"X-RapidAPI-Key": sports_data_api_key,
	"X-RapidAPI-Host": "api-nba-v1.p.rapidapi.com"
}

response_dates = requests.get(url, headers=headers, params=querystring)
response_nuggets_dates = response_dates.json()
print(json.dumps(response_nuggets_dates, indent=4))
nuggets_calendar_df = pd.json_normalize(response_nuggets_dates['response']).reset_index(drop=True)
nuggets_calendar_df = nuggets_calendar_df['date.start'].reset_index().rename(columns={'index':'game','date.start':'date'})
nuggets_calendar_df

{
    "get": "games/",
    "parameters": {
        "season": "2023",
        "team": "9"
    },
    "errors": [],
    "results": 99,
    "response": [
        {
            "id": 12496,
            "league": "standard",
            "season": 2023,
            "date": {
                "start": "2023-10-11T02:00:00.000Z",
                "end": null,
                "duration": null
            },
            "stage": 1,
            "status": {
                "clock": null,
                "halftime": false,
                "short": 3,
                "long": "Finished"
            },
            "periods": {
                "current": 4,
                "total": 4,
                "endOfPeriod": false
            },
            "arena": {
                "name": "Footprint Center",
                "city": "Phoenix",
                "state": "AZ",
                "country": null
            },
            "teams": {
                "visitors": {
                    "id": 9,
           

,game,date
0,0,2023-10-11T02:00:00.000Z
1,1,2023-10-13T00:00:00.000Z
2,2,2023-10-16T01:00:00.000Z
3,3,2023-10-18T02:30:00.000Z
4,4,2023-10-20T02:00:00.000Z
...,...,...
94,94,2024-05-11T01:30:00.000Z
95,95,2024-05-13T00:00:00.000Z
96,96,2024-05-15T02:30:00.000Z
97,97,2024-05-17T00:30:00.000Z


In [10]:
nuggets_calendar_df = pd.json_normalize(response_nuggets_dates['response']).reset_index(drop=True)
nuggets_calendar_df = nuggets_calendar_df['date.start'].reset_index().rename(columns={'index':'game','date.start':'date'})
nuggets_calendar_df

,game,date
0,0,2023-10-11T02:00:00.000Z
1,1,2023-10-13T00:00:00.000Z
2,2,2023-10-16T01:00:00.000Z
3,3,2023-10-18T02:30:00.000Z
4,4,2023-10-20T02:00:00.000Z
...,...,...
94,94,2024-05-11T01:30:00.000Z
95,95,2024-05-13T00:00:00.000Z
96,96,2024-05-15T02:30:00.000Z
97,97,2024-05-17T00:30:00.000Z


In [7]:
nuggets_df_csv = nuggets_df.to_csv('Resources/Denver Nuggets.csv', index=False)

In [12]:
nuggets_calendar_csv = nuggets_calendar_df.to_csv('Resources/nuggets schedule.csv', index=False)